In [7]:
import pandas as pd
import datetime as dt
from mongo import Mongo
from FinMind.data import DataLoader
import sys
from dotenv import load_dotenv
import os
def config():
    load_dotenv()


    
config()
stock_id_list = pd.read_csv('上市公司基本資料.csv')['公司代號'].to_list()
stock_id_list

#if life time expired error restart vscode
repo = Mongo('trading_bot',collection='stock_price')



dl = DataLoader()
dl.login_by_token(api_token=os.getenv('FINMIND_API_KEY'))
dl.login(user_id=os.getenv('FINMIND_USER_ID'),password=os.getenv('FINMIND_PASSWORD'))


2023-06-21 15:37:16.822 | INFO     | FinMind.data.finmind_api:login:87 - Login success


True

In [12]:
from FinMind.data import DataLoader
import datetime as dt
import time
import sys

class stock_price_scrapper:
    def  __init__(self, stock_id_list= pd.read_csv('上市公司基本資料.csv')['公司代號'].to_list(), repo = Mongo(db='trading_bot',collection='stock_price')):#, config=__config()):
        self.stock_id_list = stock_id_list
        self.repo = repo
        #login to finmind
        self.dl = DataLoader()
        self.dl.login_by_token(api_token=os.getenv('FINMIND_API_KEY'))
        self.dl.login(user_id=os.getenv('FINMIND_USER_ID'),password=os.getenv('FINMIND_PASSWORD'))

    #     self.__config = config
    #     self.dl.login_by_token(api_token=self.__config.FINMIND_API_KEY)
    #     self.dl.login(user_id=self.__config.FINMIND_USER_ID,password=self.__config.FINMIND_PASSWORD

    def send_to_repo(self,df, repo):
        df.rename(columns={'date':'Timestamp'}, inplace=True)
        for _, row in df.iterrows():
            repo.send_document(row.to_dict())

    def update_data(self):
        repo = Mongo('trading_bot', 'stock_price')
        
        print('sending started')
        # Get the current day and weekday
        today = dt.datetime.now().strftime('%Y-%m-%d')
        weekday = dt.datetime.now().weekday()

        for stock_id in stock_id_list:
            try:
                stock_id = str(stock_id)
                df = repo.get_data_by_stock_id(str(stock_id))

                # If there is data in MongoDB, add new data
                if not df.empty:
                    latest_date = (pd.to_datetime(df['Timestamp'].iloc[-1]) + pd.DateOffset(days=1)).strftime('%Y-%m-%d')
                    
                    if latest_date == today and weekday < 5:
                        print(f'{stock_id} is up-to-date')
                        continue
                    
                    if weekday >= 5:
                        print("It's the weekend! Exiting the script.")
                        sys.exit()

                    stock_data = dl.taiwan_stock_daily(stock_id=stock_id, start_date=(latest_date))

                # If there is no data in the MongoDB, add data from the beginning of 2013
                else:
                    print(f'Did not find the {stock_id} in the database. Sending data beginning from 2013.')
                    stock_data = dl.taiwan_stock_daily(stock_id=stock_id, start_date='2013-01-01')

                self.send_to_repo(stock_data, repo)
                print(f'Sent {stock_id} to trading_bot stock_price')

            except Exception as e:
                print(e)
                print('Wait 1 hour and try again')
                break


In [14]:
repo = Mongo('trading_bot',collection='stock_price')
repo.get_data_by_stock_id('1101')


,Timestamp,stock_id,Trading_Volume,Trading_money,open,max,min,close,spread,Trading_turnover
0,2013-01-02,1101,6374614,248393685,38.95,39.10,38.65,39.00,0.10,1956
1,2013-01-03,1101,9710784,378361218,39.50,39.50,38.75,38.85,-0.15,2412
2,2013-01-04,1101,8682170,338671929,39.40,39.45,38.60,39.00,0.15,3011
3,2013-01-07,1101,5067416,196594008,39.10,39.10,38.65,38.90,-0.10,1500
4,2013-01-08,1101,6454905,249593429,38.90,39.10,38.20,38.50,-0.40,2543
...,...,...,...,...,...,...,...,...,...,...
2555,2023-06-15,1101,12380474,474152763,38.35,38.40,38.20,38.35,0.00,6313
2556,2023-06-16,1101,39153380,1520424959,38.40,39.20,38.30,38.70,0.35,13057
2557,2023-06-19,1101,12887442,499013141,38.75,38.95,38.60,38.70,0.00,6972
2558,2023-06-20,1101,17678920,681580924,38.70,38.75,38.45,38.55,-0.15,5837


In [13]:
stock_price_scrapper = stock_price_scrapper()
stock_price_scrapper.update_data()

2023-06-21 15:40:23.416 | INFO     | FinMind.data.finmind_api:login:87 - Login success


sending started
Sent 1101 to trading_bot stock_price
Sent 1102 to trading_bot stock_price
Sent 1103 to trading_bot stock_price
Sent 1104 to trading_bot stock_price
Sent 1108 to trading_bot stock_price
Sent 1109 to trading_bot stock_price
Sent 1110 to trading_bot stock_price
Sent 1201 to trading_bot stock_price
Sent 1203 to trading_bot stock_price
Sent 1210 to trading_bot stock_price
Sent 1213 to trading_bot stock_price
Sent 1215 to trading_bot stock_price
Sent 1216 to trading_bot stock_price
Sent 1217 to trading_bot stock_price
Sent 1218 to trading_bot stock_price
Sent 1219 to trading_bot stock_price
Sent 1220 to trading_bot stock_price
Sent 1225 to trading_bot stock_price
Sent 1227 to trading_bot stock_price
Sent 1229 to trading_bot stock_price
Sent 1231 to trading_bot stock_price
Sent 1232 to trading_bot stock_price
Sent 1233 to trading_bot stock_price
Sent 1234 to trading_bot stock_price
Sent 1235 to trading_bot stock_price
Sent 1236 to trading_bot stock_price
Sent 1256 to trading_b

KeyboardInterrupt: 

In [ ]:
repo.get_data_by_stock_id('1103')


,Timestamp,stock_id,Trading_Volume,Trading_money,open,max,min,close,spread,Trading_turnover
0,2013-01-02,1102,2856071,107091777,37.40,37.60,37.35,37.45,0.10,1162
1,2013-01-03,1102,2193238,82180614,37.50,37.50,37.35,37.50,0.05,885
2,2013-01-04,1102,3016732,112535907,37.50,37.55,37.00,37.20,-0.30,1111
3,2013-01-07,1102,2796643,103115041,37.00,37.05,36.65,37.00,-0.20,1046
4,2013-01-08,1102,1535644,56539910,36.95,37.00,36.75,36.85,-0.15,704
...,...,...,...,...,...,...,...,...,...,...
2542,2023-05-29,1102,4356406,191149272,43.60,44.15,43.60,43.85,-0.05,2684
2543,2023-05-30,1102,4436916,195655289,44.05,44.35,43.85,44.05,0.20,2779
2544,2023-05-31,1102,6156105,270990093,44.05,44.35,43.95,43.95,-0.10,1869
2545,2023-06-01,1102,3619825,159601846,44.15,44.30,43.90,44.05,0.10,2317


In [53]:
stock_data = dl.taiwan_stock_tick(stock_id=2330, date='2023-05-22')
stock_data

,date,stock_id,deal_price,volume,Time,TickType
0,2023-05-22,2330,532.0,1719,09:00:02.428250,0
1,2023-05-22,2330,533.0,5,09:00:02.560288,1
2,2023-05-22,2330,532.0,2,09:00:02.624061,2
3,2023-05-22,2330,533.0,1,09:00:02.898222,1
4,2023-05-22,2330,533.0,4,09:00:02.930319,1
...,...,...,...,...,...,...
4281,2023-05-22,2330,530.0,1,13:24:55.074452,2
4282,2023-05-22,2330,530.0,1,13:24:55.222547,2
4283,2023-05-22,2330,531.0,1,13:24:57.515188,1
4284,2023-05-22,2330,531.0,3254,13:30:00.000000,1


In [ ]:
repo = Mongo('trading_bot',collection='stock_price')
repo.get_all_data()

In [10]:
pd.DataFrame(repo.collection.find({}))

,_id,Timestamp,stock_id,Trading_Volume,Trading_money,open,max,min,close,spread,Trading_turnover
0,64730bee20ba70069d744408,2012-01-02,1101,7028883,245403706,35.00,35.05,34.60,34.70,-0.30,1715
1,64730bee20ba70069d744409,2012-01-03,1101,13231764,465373269,34.95,35.40,34.90,35.40,0.70,4470
2,64730bee20ba70069d74440a,2012-01-04,1101,7270291,258736686,35.80,35.80,35.45,35.60,0.20,2577
3,64730bee20ba70069d74440b,2012-01-05,1101,15263996,540492635,35.80,35.80,35.00,35.60,0.00,2949
4,64730bee20ba70069d74440c,2012-01-06,1101,16463826,580971564,35.60,35.65,34.95,34.95,-0.65,3512
...,...,...,...,...,...,...,...,...,...,...,...
118525,6473156e20ba70069d761305,2012-11-26,1324,28008,294732,10.50,10.60,10.50,10.55,-0.05,15
118526,6473156e20ba70069d761306,2012-11-27,1324,14000,147050,10.50,10.60,10.45,10.60,0.05,11
118527,6473156e20ba70069d761307,2012-11-28,1324,17400,182470,10.50,10.55,10.45,10.55,-0.05,13
118528,6473156e20ba70069d761308,2012-11-29,1324,26880,284046,10.60,10.65,10.50,10.60,0.05,17


In [18]:
print(repo.get_data_by_stock_id('0000').empty)

True


In [5]:
import sys
import datetime as dt

def send_to_repo(df, repo):
    df.rename(columns={'date':'Timestamp'}, inplace=True)
    for _, row in df.iterrows():
        repo.send_document(row.to_dict())

def update_data():
    repo = Mongo('trading_bot', 'stock_price')

    # Estimate the time to finish
    hour_to_finish = len(stock_id_list) / 600
    print(f'It will take approximately {hour_to_finish} hours to run')
    
    # Get the current day and weekday
    today = dt.datetime.now().strftime('%Y-%m-%d')
    weekday = dt.datetime.now().weekday()

    for stock_id in stock_id_list:
        print(f'Sending {stock_id} to the database')

        try:
            stock_id = str(stock_id)
            df = repo.get_data_by_stock_id(str(stock_id))

            # If there is data in MongoDB, add new data
            if not df.empty:
                latest_date = (pd.to_datetime(df['Timestamp'].iloc[-1]) + pd.DateOffset(days=1)).strftime('%Y-%m-%d')
                
                if latest_date == today and weekday < 5:
                    print(f'{stock_id} is up-to-date')
                    continue
                
                if weekday >= 5:
                    print("It's the weekend! Exiting the script.")
                    sys.exit()

                stock_data = dl.taiwan_stock_daily(stock_id=stock_id, start_date=(latest_date))

            # If there is no data in the MongoDB, add data from the beginning of 2013
            else:
                print(f'Did not find the {stock_id} in the database. Sending data beginning from 2013.')
                stock_data = dl.taiwan_stock_daily(stock_id=stock_id, start_date='2013-01-01')

            dfs = pd.concat([df,stock_data])
            send_to_repo(stock_data, repo)
            print(f'Sent {stock_id} to trading_bot stock_price')

        except Exception as e:
            print(e)
            print('Wait 1 hour and try again')
            
            try:
                time.sleep(3600)
            except KeyboardInterrupt:
                print("Interrupted by user")
                return


In [2]:
repo = Mongo('trading_bot', 'stock_price')
repo.get_data_by_stock_id('2330')

,Timestamp,stock_id,Trading_Volume,Trading_money,open,max,min,close,spread,Trading_turnover
0,2013-01-02,2330,40647121,4021304950,97.6,99.9,97.1,99.6,2.6,14353
1,2013-01-03,2330,44335846,4472175100,100.5,102.0,100.0,101.0,1.4,12616
2,2013-01-04,2330,39502577,3980403777,100.5,101.5,100.0,101.5,0.5,10276
3,2013-01-07,2330,42328657,4237769552,101.0,101.0,99.1,100.5,-1.0,9042
4,2013-01-08,2330,31218717,3100841808,99.6,100.0,98.9,99.7,-0.8,9463
...,...,...,...,...,...,...,...,...,...,...
2540,2023-05-25,2330,68360924,36976525917,542.0,545.0,537.0,543.0,18.0,80930
2541,2023-05-26,2330,104924623,59322536825,568.0,568.0,563.0,566.0,23.0,131612
2542,2023-05-29,2330,58813792,33454887785,574.0,574.0,564.0,568.0,2.0,52250
2543,2023-05-30,2330,47490594,26861389177,566.0,568.0,563.0,566.0,-2.0,32956


In [ ]:
repo = Mongo(db='trading_bot', collection='stock_price')
repo.get_data_by_stock_id('1101')

,Timestamp,stock_id,Trading_Volume,Trading_money,open,max,min,close,spread,Trading_turnover
0,2012-01-02,1101,7028883,245403706,35.00,35.05,34.60,34.70,-0.30,1715
1,2012-01-03,1101,13231764,465373269,34.95,35.40,34.90,35.40,0.70,4470
2,2012-01-04,1101,7270291,258736686,35.80,35.80,35.45,35.60,0.20,2577
3,2012-01-05,1101,15263996,540492635,35.80,35.80,35.00,35.60,0.00,2949
4,2012-01-06,1101,16463826,580971564,35.60,35.65,34.95,34.95,-0.65,3512
...,...,...,...,...,...,...,...,...,...,...
12704,2023-05-25,1101,30414582,1146187680,37.70,38.05,37.50,38.05,0.20,13036
12705,2023-05-26,1101,21233325,802585508,37.80,38.00,37.60,38.00,-0.05,9510
12706,2023-05-29,1101,26028333,987145356,38.05,38.15,37.75,38.15,0.15,11969
12707,2023-05-29,1101,26028333,987145356,38.05,38.15,37.75,38.15,0.15,11969
